<a href="https://colab.research.google.com/github/Moukthika1253/NaiveBayesClassifier-Rotten-Tomato-reviews/blob/main/NBC_Rotten_tomato_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
rottenTomato_reviews=pd.read_csv('/content/rt_reviews.csv',encoding='latin-1')

In [34]:
freshness=list(rottenTomato_reviews['Freshness'])
pos_reviews=0
neg_reviews=0
pos_words=0
neg_words=0
word_dict=defaultdict(lambda:[0,0])

**Counting total fresh and rotten reviews**

In [36]:
for i in range(len(list(rottenTomato_reviews['Freshness']))):
  if freshness[i] == 'fresh':
    pos_reviwes=pos_reviews+1
  else:
    neg_reviews=neg_reviews+1

 **Pre-processing**

In [37]:
stop_words=stopwords.words('english')
word_lemmatizer=WordNetLemmatizer()
stemming=PorterStemmer()
for i,review in enumerate(list(rottenTomato_reviews['Review'])):
 for word in word_tokenize(review):
  word=word.lower()
  word= word_lemmatizer.lemmatize(word)
  word=stemming.stem(word)
  if (word not in string.punctuation) and (word not in stop_words):
   if freshness[i] == 'fresh':
    word_dict[word][1]+=1
    pos_words+=1
  else:
    word_dict[word][0]+=1
    neg_words=1

**Removing rare words**

In [42]:
dict_copy=word_dict.copy()
for k,v in dict_copy.items():
  if(v[1]<5):
    del[word_dict[k]]